# Instalação

In [ ]:
#!pip install xcompact3d-toolbox

# Lendo e salvando arquivos

Esse tutorial apresenta diferentes formas com que se pode carregar arquivos binários do disco, após rodar uma simulação com [XCompact3d](https://github.com/xcompact3d/Incompact3d). Além disso, são apresentados algumas opções para salvar resultados de pós-processamento.

## Preparação

Aqui se prepara o banco de dados para esse notebook, para que possa ser reproduzido na máquina local ou na nuvem.

O primeiro passo é para importar as bibliotecas para esse notebook.

In [1]:
import warnings

import numpy as np
import xarray as xr

import xcompact3d_toolbox as x3d

Será utilizado como exemplo o escoamento ao redor de um cilindro como [banco de dados](https://github.com/fschuch/xcompact3d_toolbox_data).
Definindo `cache=True` pode-se definir um local para salvar os dados no seu computador, `cache_dir="./example/"`.

In [2]:
cylinder_ds, prm = x3d.tutorial.open_dataset("cylinder", cache=True, cache_dir="./example/")

Nesse momento é bom verificar as informações do banco de dados.

In [3]:
cylinder_ds.info()

xarray.Dataset {
dimensions:
	i = 2 ;
	x = 257 ;
	y = 128 ;
	t = 201 ;

variables:
	float32 u(i, x, y, t) ;
	float32 pp(x, y, t) ;
	float32 epsi(x, y) ;
	float64 x(x) ;
	float64 y(y) ;
	float64 t(t) ;
	<U1 i(i) ;

// global attributes:
	:xcompact3d_version = v3.0-397-gff531df ;
	:xcompact3d_toolbox_version = 1.0.1 ;
	:url = https://github.com/fschuch/xcompact3d_toolbox_data ;
	:dataset_license = MIT License ;
}

Os dados são organizados em um [xarray.Dataset](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) com as variáveis `u` (campo de velocidade), `pp` (pressão) e `epsi` (descrição da geométria), suas coordenadas (`x`, `y`, `t` e `i`) e alguns atributos como a versão do Xcompact3d e a `url` onde pode-se encontrar o banco de dados.

No próximo bloco se configura a toolbox e alguns atributos do banco de dados, para que se possa salvar todos os arquivos binários no disco. Os comandos a baixo seram descritos melhor adiante.

In [4]:
x3d.param["mytype"] = np.float32

prm.dataset.set(data_path="./data/", drop_coords="z")

cylinder_ds.u.attrs["file_name"] = "u"
cylinder_ds.pp.attrs["file_name"] = "pp"
cylinder_ds.epsi.attrs["file_name"] = "epsilon"

prm.write("input.i3d")

prm.dataset.write(cylinder_ds)

prm.dataset.write_xdmf("xy-planes.xdmf")

del cylinder_ds, prm

ux:   0%|          | 0/201 [00:00<?, ?it/s]

uy:   0%|          | 0/201 [00:00<?, ?it/s]

pp:   0%|          | 0/201 [00:00<?, ?it/s]

xy-planes.xdmf:   0%|          | 0/201 [00:00<?, ?it/s]

Após essas linhas de código os arquivos são organizados dessa forma:

```
tutorial
│   computing_and_plotting.ipynb
│   io.ipynb
│   input.i3d
│   parameters.ipynb
│   xy-planes.xdmf
│
└─── data
│       │   epsilon.bin
│       │   pp-000.bin
│       │   pp-001.bin
│       │   ...
│       │   pp-199.bin
│       │   pp-200.bin
│       │   ux-000.bin
│       │   ux-001.bin
│       │   ...
│       │   ux-199.bin
│       │   ux-200.bin
│       │   uy-000.bin
│       │   uy-001.bin
│       │   ...
│       │   uy-199.bin
│       │   uy-200.bin
│       │   uz-000.bin
│       │   uz-001.bin
│       │   ...
│       │   uz-199.bin
│       │   uz-200.bin
│
└─── example
│       │   cylinder.nc
```

É bem próximo o que é gerado após rodar a simulação.

## Por que xarray?

As estruturas de dados são providas por [xarray](http://docs.xarray.dev/en/stable/index.html), que atribui rótulos de dimensão, coordenadas e atributos sob NumPy arrays. E utiliza [dask](https://dask.org/) para computação paralela.
O objetivo é acelerar o desenvolvimento e customização do pós-processamento.

Adicionamente, xcompact3d-toolbox prove funcionalidades para [DataArray](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.array.X3dDataArray) e [Dataset](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.array.X3dDataset).

Mais detalhes da biblioteca [xarray](http://docs.xarray.dev/en/stable/index.html) estão disponíveis em [Overview: Why xarray?](http://docs.xarray.dev/en/stable/getting-started-guide/why-xarray.html) e [Quick overview](http://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html).

## Objetos xarray sobre demanda

Como primeiro etapa do pós-processamento, se carrega o arquivo com os parâmetros da simulação.

In [5]:
prm = x3d.Parameters(loadfile="input.i3d")

Para salvar espaço no disco, o banco de dados foi convertido de precisão dupla para precisão simples, isso pode ser feito com:

In [6]:
x3d.param["mytype"] = np.float32

A ferramenta permite ler arquivos com diferentes extensões. Para mais detalhes acesse [filename properties](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.FilenameProperties). Para carregar o banco de dados baixado, pode-se confirgurar a ferramenta com o comando a baixo:

In [7]:
prm.dataset.filename_properties.set(
    separator="-",
    file_extension=".bin",
    number_of_digits=3,
)

Agora se especifica os parâmetros do banco de dados, como a localização do arquivo, se alguma coordenada pode ser ignorada (`drop_coords` permite excluir uma coordenada, como exemplo, excluir a coordenada `z` para trabalhar com o plano `xy`), o número de passos de tempo `snapshot_counting` e o tamnho do passo de tempo`snapshot_step`. Para mais detalhes acesse a documentação do módulo [dataset](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset).
Para esse exemplo se define os seguintes parâmetros:

In [8]:
prm.dataset.set(
    data_path="./data/",
    drop_coords="z",
    snapshot_counting="ilast",
    snapshot_step="ioutput",
)

Pode-se checar o [tamanho do banco de dados](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset.__len__) com:

In [9]:
len(prm.dataset)

201

Isso significa que há 201 snapshots indo de 0 (i.g., `ux-000.bin`) a 200 (i.g., `ux-200.bin`).

Pode-se carregar um arquivo binário com:

In [10]:
epsilon = prm.dataset.load_array("./data/epsilon.bin", add_time=False)

Observe que [load_array](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset.load_array) necessita o caminho completo e se usa `add_time=False` porque esse array não envolve tempo.

Pode-se visualizar na tela:

In [11]:
epsilon

<xarray.DataArray (x: 257, y: 128)> Size: 132kB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
  * x        (x) float32 1kB 0.0 0.07812 0.1562 0.2344 ... 19.84 19.92 20.0
  * y        (y) float32 512B 0.0 0.09375 0.1875 0.2812 ... 11.72 11.81 11.91

Fazendo a mesma coisa, agora para o campo de velocidade na direção x, `ux`, e usando `add_time=True`:

In [12]:
ux = prm.dataset.load_array("./data/ux-100.bin", add_time=True)

Agora `t` é uma coordenada e seu valor adimensional é automaticamente calculado como `75.0`:

In [13]:
ux

<xarray.DataArray 'ux' (x: 257, y: 128, t: 1)> Size: 132kB
array([[[1.        ],
        [1.        ],
        [1.        ],
        ...,
        [1.        ],
        [1.        ],
        [1.        ]],

       [[1.0000466 ],
        [0.99996716],
        [1.0000466 ],
        ...,
        [0.9999681 ],
        [1.0000459 ],
        [0.9999675 ]],

       [[1.0000602 ],
        [1.0000144 ],
        [1.0000602 ],
        ...,
...
        ...,
        [1.0140737 ],
        [1.0142432 ],
        [1.0144366 ]],

       [[1.0146521 ],
        [1.0148891 ],
        [1.0151588 ],
        ...,
        [1.0141058 ],
        [1.0142633 ],
        [1.0144445 ]],

       [[1.0146475 ],
        [1.0148702 ],
        [1.0151254 ],
        ...,
        [1.014144  ],
        [1.0142874 ],
        [1.014454  ]]], dtype=float32)
Coordinates:
  * x        (x) float32 1kB 0.0 0.07812 0.1562 0.2344 ... 19.84 19.92 20.0
  * y        (y) float32 512B 0.0 0.09375 0.1875 0.2812 ... 11.72 11.81 11.91
  * t        (t) float32 4B 75.0

Também pode-se carregar a [serie temporal](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset.load_time_series), completa para uma variável se houver espaço suficiente na memória, ou simples por:

In [14]:
ux = prm.dataset["ux"]

./data/ux-???.bin:   0%|          | 0/201 [00:00<?, ?it/s]

Pode-se visualizar o banco de dados na tela com:

In [15]:
ux

<xarray.DataArray 'ux' (x: 257, y: 128, t: 201)> Size: 26MB
array([[[0.9999885 , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [0.9999782 , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [0.9999907 , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        ...,
        [1.0000122 , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [0.99999744, 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [0.9999945 , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ]],

       [[0.99999535, 1.0000458 , 1.0000486 , ..., 1.0000464 ,
         1.0000468 , 1.0000476 ],
        [1.0000107 , 0.99997103, 0.99996907, ..., 0.9999675 ,
         0.99996674, 0.9999665 ],
        [1.0000069 , 1.0000455 , 1.0000486 , ..., 1.0000459 ,
         1.0000468 , 1.0000478 ],
...
        [0.9999908 , 1.0001078 , 1.000192  , ..., 1.0155317 ,
         1.0153327 , 1.0146557 ],
        [0.9999987 , 1.0001054 , 1.0001911 , ..., 1.0152053 ,
         1.0150691 , 1.0146141 ],
        [0.9999874 , 1.0000997 , 1.0001901 , ..., 1.0149139 ,
         1.0148364 , 1.0145978 ]],

       [[1.0000043 , 1.0000877 , 1.0001792 , ..., 1.0146514 ,
         1.0146394 , 1.0146087 ],
        [1.0000119 , 1.0000889 , 1.0001761 , ..., 1.014433  ,
         1.0144511 , 1.0146273 ],
        [1.0000004 , 1.0000932 , 1.000174  , ..., 1.0142416 ,
         1.0142918 , 1.0146769 ],
        ...,
        [1.0000123 , 1.0000978 , 1.0001832 , ..., 1.0154953 ,
         1.0153852 , 1.0147215 ],
        [0.99998474, 1.000096  , 1.0001816 , ..., 1.0151795 ,
         1.0151051 , 1.0146575 ],
        [0.99999154, 1.0000918 , 1.0001806 , ..., 1.0148991 ,
         1.0148568 , 1.0146192 ]]], dtype=float32)
Coordinates:
  * x        (x) float32 1kB 0.0 0.07812 0.1562 0.2344 ... 19.84 19.92 20.0
  * y        (y) float32 512B 0.0 0.09375 0.1875 0.2812 ... 11.72 11.81 11.91
  * t        (t) float32 804B 0.0 0.75 1.5 2.25 3.0 ... 147.8 148.5 149.2 150.0

Pode-se carregar cada array em uma variável diferente:

In [16]:
ux = prm.dataset["ux"]
uy = prm.dataset["uy"]
pp = prm.dataset["pp"]

./data/ux-???.bin:   0%|          | 0/201 [00:00<?, ?it/s]

./data/uy-???.bin:   0%|          | 0/201 [00:00<?, ?it/s]

./data/pp-???.bin:   0%|          | 0/201 [00:00<?, ?it/s]

Ou organizar vários arrays em um banco de dados:

In [17]:
# create an empty dataset
ds = xr.Dataset()

# populate it
for var in ["ux", "uy", "pp"]:
    ds[var] = prm.dataset[var]

# show on the screen
ds

./data/ux-???.bin:   0%|          | 0/201 [00:00<?, ?it/s]

./data/uy-???.bin:   0%|          | 0/201 [00:00<?, ?it/s]

./data/pp-???.bin:   0%|          | 0/201 [00:00<?, ?it/s]

<xarray.Dataset> Size: 79MB
Dimensions:  (x: 257, y: 128, t: 201)
Coordinates:
  * x        (x) float32 1kB 0.0 0.07812 0.1562 0.2344 ... 19.84 19.92 20.0
  * y        (y) float32 512B 0.0 0.09375 0.1875 0.2812 ... 11.72 11.81 11.91
  * t        (t) float32 804B 0.0 0.75 1.5 2.25 3.0 ... 147.8 148.5 149.2 150.0
Data variables:
    ux       (x, y, t) float32 26MB 1.0 1.0 1.0 1.0 ... 1.015 1.015 1.015 1.015
    uy       (x, y, t) float32 26MB 9.98e-06 8.496e-08 ... -0.0005357 0.003209
    pp       (x, y, t) float32 26MB 0.0 0.03264 0.03613 ... 0.03922 0.03859

A mesma solução pode ser escrita em uma única linha de código:

```python
ds = xr.Dataset({var: prm.dataset[var] for var in "ux uy pp".split()})
```

É possível carregar todas as variáveis de um único snapshot com [load_snapshot](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset.load_snapshot) ou:

In [18]:
snapshot = prm.dataset[100]

Se obtém um [xarray.Dataset](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) com todas as variáveis com as suas coordenadas. Cada variável pode ser acessada individualmente com a notação por ponto (i.g., `snapshot.pp`, `snapshot.ux`, `snapshot.uy`) ou por colchete (i.g., `snapshot["pp"]`, `snapshot["ux"]`, `snapshot["uy"]`).

In [19]:
snapshot

<xarray.Dataset> Size: 396kB
Dimensions:  (x: 257, y: 128, t: 1)
Coordinates:
  * x        (x) float32 1kB 0.0 0.07812 0.1562 0.2344 ... 19.84 19.92 20.0
  * y        (y) float32 512B 0.0 0.09375 0.1875 0.2812 ... 11.72 11.81 11.91
  * t        (t) float32 4B 75.0
Data variables:
    pp       (x, y, t) float32 132kB 0.05232 0.05219 0.05243 ... 0.03986 0.03989
    ux       (x, y, t) float32 132kB 1.0 1.0 1.0 1.0 ... 1.014 1.014 1.014 1.014
    uy       (x, y, t) float32 132kB 3.407e-07 1.503e-07 ... 0.007724 0.007703

Pode-se usar [slice](https://docs.python.org/3/library/functions.html#slice) para selecionar uma seção do banco de dados para calcular a [média temporal](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.mean.html):

In [20]:
time_averaged = prm.dataset[-100:].mean("t")
time_averaged

<xarray.Dataset> Size: 396kB
Dimensions:  (x: 257, y: 128)
Coordinates:
  * x        (x) float32 1kB 0.0 0.07812 0.1562 0.2344 ... 19.84 19.92 20.0
  * y        (y) float32 512B 0.0 0.09375 0.1875 0.2812 ... 11.72 11.81 11.91
Data variables:
    pp       (x, y) float32 132kB 0.05351 0.05335 0.05356 ... 0.03886 0.03886
    ux       (x, y) float32 132kB 1.0 1.0 1.0 1.0 ... 1.015 1.015 1.015 1.015
    uy       (x, y) float32 132kB -6.206e-09 2.081e-09 ... -6.504e-05 -6.531e-05

Também pode-se carregar todos o snapshots com a seguinte notação:

In [21]:
prm.dataset[:]

<xarray.Dataset> Size: 79MB
Dimensions:  (x: 257, y: 128, t: 201)
Coordinates:
  * x        (x) float32 1kB 0.0 0.07812 0.1562 0.2344 ... 19.84 19.92 20.0
  * y        (y) float32 512B 0.0 0.09375 0.1875 0.2812 ... 11.72 11.81 11.91
  * t        (t) float32 804B 0.0 0.75 1.5 2.25 3.0 ... 147.8 148.5 149.2 150.0
Data variables:
    pp       (x, y, t) float32 26MB 0.0 0.03264 0.03613 ... 0.03922 0.03859
    ux       (x, y, t) float32 26MB 1.0 1.0 1.0 1.0 ... 1.015 1.015 1.015 1.015
    uy       (x, y, t) float32 26MB 9.98e-06 8.496e-08 ... -0.0005357 0.003209

Para o caso que não aja memória suficiente para carregar toda simulação, pode-se iter sobre todos os snapshots um por um:

In [22]:
for ds in prm.dataset:
    # Calcular a vorticidade como exemplo
    vort = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")

Outra opção viável é usar `reversed(prm.dataset)` para iterar na ordem inversa.

Também pode-se selecionar um intervalo de snapshots carregados um por um:

In [23]:
for ds in prm.dataset(100, 200, 1):
    # Calcular a vorticidade como exemplo
    vort = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")

In [24]:
# Resultado da última interação
vort

<xarray.DataArray (y: 128, t: 1, x: 257)> Size: 132kB
array([[[-0.00268999,  0.00181677,  0.0001196 , ...,  0.00324146,
          0.00302246, -0.01646465]],

       [[-0.0034743 , -0.00406752, -0.00344934, ...,  0.0029434 ,
          0.00254392, -0.01612649]],

       [[-0.00303078, -0.00340789, -0.0030852 , ...,  0.0026302 ,
          0.0024012 , -0.01718407]],

       ...,

       [[-0.00178402, -0.00190772, -0.00161242, ...,  0.00415556,
          0.00375753, -0.01497194]],

       [[-0.00232817, -0.00317189, -0.00255065, ...,  0.00385816,
          0.00362989, -0.01590095]],

       [[-0.00263563,  0.00207518,  0.00038911, ...,  0.00355052,
          0.00316822, -0.01547501]]], dtype=float32)
Coordinates:
  * x        (x) float32 1kB 0.0 0.07812 0.1562 0.2344 ... 19.84 19.92 20.0
  * y        (y) float32 512B 0.0 0.09375 0.1875 0.2812 ... 11.72 11.81 11.91
  * t        (t) float32 4B 149.2

## Writing the results to binary files

In the last example we computed the vorticity but did nothing with it. This time, let's write it to the disc using [write](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.io.Dataset.write):

In [ ]:
for ds in prm.dataset:
    vort = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")
    prm.dataset.write(data=vort, file_prefix="w3")

The example above works for a [xarray.DataArray](http://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray). We can do it for a [xarray.Dataset](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) as well, but with one key difference. Only the arrays with an attribute called `file_name` will be written. It is done to avoid overwriting the base fields (`ux`, `uy`, `uz`, ...) by accident.

Let's rewrite the previous example to store `vort` in the dataset `ds`. We set an attribute `file_name` to `w3`, so the arrays will be written as `w3-000.bin`, `w3-001.bin`, `w3-002.bin`, etc.

We are also suppressing warnings, because the application will tell us it can not save `pp`, `ux` and `uy`, since they do not have a `file_name`. But in fact, we do not want to rewrite them anyway.

See the code:

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    for ds in prm.dataset:
        ds["vort"] = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")
        ds["vort"].attrs["file_name"] = "w3"
        prm.dataset.write(ds)

The method [prm.dataset.write()](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset.write) writes the files as raw binaries in the same way that [XCompact3d](https://github.com/xcompact3d/Incompact3d) would do. It means you can read them at the flow solver and also process them on any other tool that you are already familiar with, including the toolbox.

For instance, we get `w3` if we load snapshot 0 again:

In [ ]:
prm.dataset[0]

### Update the xdmf file

After computing and writing new results to the disc, you can open them on any external tools, like Paraview or Visit. You can update the xdmf file to include the recently computed `w3`. See the code:

In [ ]:
prm.dataset.write_xdmf("xy-planes.xdmf")

## Other formats

Xarray objects can be exported to many other formats, depending on your needs.

For instance, [xarray.DataArray](http://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray) and [xarray.Dataset](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) can be written as [netCDF](http://docs.xarray.dev/en/stable/user-guide/io.html). In this way, they will keep all dimensions, coordinates, and attributes. This format is easier to handle and share because the files are self-sufficient. It is the format used to download the dataset used in this tutorial, and it is a good alternative to use when sharing the results of your research.

Just to give you an estimation about the disk usage, the size of the dataset `cylinder.nc` that we downloaded for this tutorial is 75.8 MB. The size of the folder `./data/` after producing the binary files in the same way that [XCompact3d](https://github.com/xcompact3d/Incompact3d) would do is 75.7 MB.

To exemplify the use of netCDF, let's take one snapshot:

In [ ]:
snapshot = prm.dataset[0]
snapshot

Now, let's include additional information for the ones that are going to use our data. You can set attributes for each array, coordinate, and also global attributes for the dataset. They are stored in a dictionary.

See the example:

In [ ]:
# Setting attributes for each coordinate
snapshot.x.attrs = {"name": "x", "long_name": "Stream-wise coordinate", "units": "-"}
snapshot.y.attrs = {"name": "y", "long_name": "Vertical coordinate", "units": "-"}
snapshot.t.attrs = {"name": "t", "long_name": "Time", "units": "-"}

# Setting attributes for each array
snapshot.ux.attrs = {"name": "ux", "long_name": "Stream-wise velocity", "units": "-"}
snapshot.uy.attrs = {"name": "y", "long_name": "Vertical velocity", "units": "-"}
snapshot.pp.attrs = {"name": "p", "long_name": "Pressure", "units": "-"}
snapshot.w3.attrs = {"name": "w3", "long_name": "Vorticity", "units": "-"}

# Setting attributes for the dataset
snapshot.attrs = {
    "title": "An example from the tutorials",
    "url": "https://docs.fschuch.com/xcompact3d_toolbox/tutorial/io.html",
    "authors": "List of names",
    "doi": "maybe a fancy doi from zenodo",
}

In [ ]:
# Setting attributes for each coordinate
snapshot.x.attrs = {"name": "x", "long_name": "Stream-wise coordinate", "units": "-"}
snapshot.y.attrs = {"name": "y", "long_name": "Vertical coordinate", "units": "-"}
snapshot.t.attrs = {"name": "t", "long_name": "Time", "units": "-"}

# Setting attributes for each array
snapshot.ux.attrs = {"name": "ux", "long_name": "Stream-wise velocity", "units": "-"}
snapshot.uy.attrs = {"name": "y", "long_name": "Vertical velocity", "units": "-"}
snapshot.pp.attrs = {"name": "p", "long_name": "Pressure", "units": "-"}
snapshot.w3.attrs = {"name": "w3", "long_name": "Vorticity", "units": "-"}

# Setting attributes for the dataset
snapshot.attrs = {
    "title": "An example from the tutorials",
    "url": "https://docs.fschuch.com/xcompact3d_toolbox/tutorial/io.html",
    "authors": "List of names",
    "doi": "maybe a fancy doi from zenodo",
}

In [ ]:
# Setting attributes for each coordinate
snapshot.x.attrs = {"name": "x", "long_name": "Stream-wise coordinate", "units": "-"}
snapshot.y.attrs = {"name": "y", "long_name": "Vertical coordinate", "units": "-"}
snapshot.t.attrs = {"name": "t", "long_name": "Time", "units": "-"}

# Setting attributes for each array
snapshot.ux.attrs = {"name": "ux", "long_name": "Stream-wise velocity", "units": "-"}
snapshot.uy.attrs = {"name": "y", "long_name": "Vertical velocity", "units": "-"}
snapshot.pp.attrs = {"name": "p", "long_name": "Pressure", "units": "-"}
snapshot.w3.attrs = {"name": "w3", "long_name": "Vorticity", "units": "-"}

# Setting attributes for the dataset
snapshot.attrs = {
    "title": "An example from the tutorials",
    "url": "https://docs.fschuch.com/xcompact3d_toolbox/tutorial/io.html",
    "authors": "List of names",
    "doi": "maybe a fancy doi from zenodo",
}

In [ ]:
# Setting attributes for each coordinate
snapshot.x.attrs = {"name": "x", "long_name": "Stream-wise coordinate", "units": "-"}
snapshot.y.attrs = {"name": "y", "long_name": "Vertical coordinate", "units": "-"}
snapshot.t.attrs = {"name": "t", "long_name": "Time", "units": "-"}

# Setting attributes for each array
snapshot.ux.attrs = {"name": "ux", "long_name": "Stream-wise velocity", "units": "-"}
snapshot.uy.attrs = {"name": "y", "long_name": "Vertical velocity", "units": "-"}
snapshot.pp.attrs = {"name": "p", "long_name": "Pressure", "units": "-"}
snapshot.w3.attrs = {"name": "w3", "long_name": "Vorticity", "units": "-"}

# Setting attributes for the dataset
snapshot.attrs = {
    "title": "An example from the tutorials",
    "url": "https://docs.fschuch.com/xcompact3d_toolbox/tutorial/io.html",
    "authors": "List of names",
    "doi": "maybe a fancy doi from zenodo",
}

Exporting it as a netCDF file:

In [ ]:
snapshot.to_netcdf("snapshot-000.nc")

Importing the netCDF file:

In [ ]:
snapshot_in = xr.open_dataset("snapshot-000.nc")

See the result, it keeps all dimensions, coordinates, and attributes:

In [ ]:
snapshot_in

We can compare them and see that their data, dimensions and coordinates are exactly the same:

In [ ]:
xr.testing.assert_equal(snapshot, snapshot_in)

Xarray is built on top of Numpy, so you can access a `numpy.ndarray` object with the property `values` (i.g., `epsilon.values`). It is compatible with `numpy.save` and many other methods from the Numpy/SciPy ecosystem (many times, you do not even need to explicitly use `.values`). See the example:

In [ ]:
np.save("epsi.npy", epsilon)
epsi_in = np.load("epsi.npy")

print(type(epsi_in))
epsi_in

You can use it for backwards compatibility with your previous post-processing tools. It is just not so effective, because we lost track of metadata like the coordinates and attributes.

If you manage to reduce the dataset's dimensions with some [integration](https://docs.xarray.dev/en/stablegenerated/xarray.Dataset.integrate.html), [mean](https://docs.xarray.dev/en/stablegenerated/xarray.Dataset.mean.html), or [selecting](https://docs.xarray.dev/en/stablegenerated/xarray.Dataset.sel.html) subsets of data, you can convert it to a [pandas.Dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) and then export it to CSV, Excel, and many other options.

For instance, let's select a vertical profile for all variables where `x = 20` and [convert it to a dataframe](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.to_dataframe.html):

In [ ]:
snapshot_in.sel(x=20.0).to_dataframe()

Now, you can refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/index.html) for more details.